In [20]:
import pandas as pd
import re
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [21]:
df=pd.read_csv('./TestData/EDSP32k_Hydv1.8.csv')
df.head()

,#SMILES,name,CAS #,Synthesis Code,Generation,Phase,Formation,Degradation,Production,Accumulation,Route,Exact Mass
0,C=O,Formaldehyde,"50-00-0, 630-08-0, 13007-92-6, 13463-39-3, 134...",1,0,0,0,0,100.00%,< 0.001%,NaN,30.01056468
1,CC1CC2C3CCC4=CC(=O)C=CC4(C)C3(F)C(O)CC2(C)C1(O...,Dexamethasone,"50-02-2, 378-44-9, 1249-18-9, 000050-02-2, 000...",2,0,0,0,0,100.00%,< 0.001%,NaN,392.1999022
2,CC(=O)OCC(=O)C1(O)CCC2C3CCC4=CC(=O)CCC4(C)C3C(...,Hydrocortisone acetate,"50-03-3, 000050-03-3",3,0,0,0,98,100.00%,< 0.001%,NaN,404.2198888
3,CC(O)=O,NaN,NaN,Carboxylic Acid Ester Hydrolysis(3):1/1,1,0,49,0,50.00%,50.00%,Carboxylic Acid Ester Hydrolysis,60.02112937
4,CC12CC(O)C3C(CCC4=CC(=O)CCC34C)C1CCC2(O)C(=O)CO,NaN,NaN,Carboxylic Acid Ester Hydrolysis(3):1/2,1,0,49,0,50.00%,50.00%,Carboxylic Acid Ester Hydrolysis,362.2093241


In [22]:
p=df[df['Generation']==0]
Parents=p[['name','CAS #','#SMILES','Synthesis Code' ]]
Parents

,name,CAS #,#SMILES,Synthesis Code
0,Formaldehyde,"50-00-0, 630-08-0, 13007-92-6, 13463-39-3, 134...",C=O,1
1,Dexamethasone,"50-02-2, 378-44-9, 1249-18-9, 000050-02-2, 000...",CC1CC2C3CCC4=CC(=O)C=CC4(C)C3(F)C(O)CC2(C)C1(O...,2
2,Hydrocortisone acetate,"50-03-3, 000050-03-3",CC(=O)OCC(=O)C1(O)CCC2C3CCC4=CC(=O)CCC4(C)C3C(...,3
5,Cortisone acetate,50-04-4,CC(=O)OCC(=O)C1(O)CCC2C3CCC4=CC(=O)CCC4(C)C3C(...,4
8,Phenobarbital,"50-06-6, 57-30-7, 000050-06-6",CCC1(C(=O)NC(=O)NC1=O)c1ccccc1,5
...,...,...,...,...
59060,2-Methoxy-4-nitrobenzenediazonium,125552-45-6\n27761-26-8,COc1cc(ccc1[N+]#N)[N+]([O-])=O,32578
59061,"(2,4-dichlorophenyl) acetate",6341-97-5,CC(=O)Oc1ccc(Cl)cc1Cl,32579
59064,"2,4-Diisocyanato-1-methylcyclohexane",10581-16-5,CC1CCC(CC1N=C=O)N=C=O,32580
59065,"dimethyl 2,2-diethylpropanedioate",27132-23-6,CCC(CC)(C(=O)OC)C(=O)OC,32581


In [23]:
#extract parent number from synthesis code and save as column 'Parent_num'
df['Parent_ID']=df['Synthesis Code'].str.extract(r"\((\d+)\)")

#group data by parent ID and create a list of unique 'Routes'
group=df.groupby(['Parent_ID'])['Route'].unique().reset_index()
group

,Parent_ID,Route
0,10001,[Lactam Hydrolysis]
1,10002,[Lactam Hydrolysis]
2,10004,[Nitrile Hydrolysis]
3,10005,[Amide Hydrolysis]
4,10007,[Carboxylic Acid Ester Hydrolysis]
...,...,...
12798,9992,[Lactam Hydrolysis]
12799,9993,[Nitrile Hydrolysis]
12800,9995,[Carboxylic Acid Ester Hydrolysis]
12801,9997,[Carboxylic Acid Ester Hydrolysis]


In [24]:
#create empty lists for each functional group 
lac=[]
imi=[]
ami=[]
ur=[]
sulf=[]
nit=[]
cu=[]


#loop through each row in group dataframe
#if there is only one elements in list for column'Route', see if the value is lactam, amide, or imide
# if so, save the parent ID to the appropriate list

for i in group.index:
    check=group['Route'][i]
    par=group['Parent_ID'][i]
    if len(check) == 1:
        if check == ['Lactam Hydrolysis']:
            lac.append(par)
        elif check == ['Imide Hydrolysis']:
            imi.append(par)
        elif check == ['Amide Hydrolysis']:
            ami.append(par)
        elif check == ['Urea Hydrolysis']:
            ur.append(par)
        elif check == ['Sulfonylurea Hydrolysis']:
            sulf.append(par)
        elif check == ['Nitrile Hydrolysis']:
            nit.append(par)
        elif check == ['Cyclic Urea Hydrolysis']:
            cu.append(par)

In [25]:
#make subsets
urea=df[df['Parent_ID'].isin(ur)==True]
sulfonylurea=df[df['Parent_ID'].isin(sulf)==True]
nitrile=df[df['Parent_ID'].isin(nit)==True]
cyclic=df[df['Parent_ID'].isin(cu)==True]
amide=df[df['Parent_ID'].isin(ami)==True]
imide=df[df['Parent_ID'].isin(imi)==True]
lactam=df[df['Parent_ID'].isin(lac)==True]

In [34]:
def ByRow(sub,parent_df):
    
    #group product smiles into a list based on parent ID, 
    temp=sub.groupby('Parent_ID')['#SMILES'].agg(list).reset_index()
    
    #seperate out product smiles into columns
    prod=pd.concat([pd.DataFrame(temp['#SMILES'].values.tolist()),temp['Parent_ID']],axis=1)
    
    #get parent information
    p=parent_df[parent_df['Synthesis Code'].isin(temp['Parent_ID'])==True]
    
    #merge parent information with product information
    new=p.merge(prod,left_on='Synthesis Code', right_on='Parent_ID')
    
    #make Parent ID first column
    new.insert(0,'Parent_ID',new.pop('Parent_ID'))
    new.drop(columns='Synthesis Code',inplace=True)
    new.rename(columns={'#SMILES': 'Parent_SMILES'},inplace=True)
    
    return new

In [35]:
u=ByRow(urea,Parents)
u

,Parent_ID,name,CAS #,Parent_SMILES,0,1,2,3
0,205,Urea,"57-13-6, 124-43-6, 124-47-0, 506-89-8, 9011-05...",NC(N)=O,N,N,None,None
1,372,"Urea, phenyl-","64-10-8, 000064-10-8",NC(=O)Nc1ccccc1,NC1=CC=CC=C1,N,None,None
2,943,"N,N'-Diethylcarbanilide",85-98-3,CCN(C(=O)N(CC)c1ccccc1)c1ccccc1,CCNC1=CC=CC=C1,CCNC1=CC=CC=C1,None,None
3,1125,Diethylcarbamazine,"90-89-1, 1642-54-2, 000090-89-1",CCN(CC)C(=O)N1CCN(C)CC1,CN1CCNCC1,CCNCC,None,None
4,1363,"N,N'-Dimethylurea","96-31-1, 000096-31-1",CNC(=O)NC,CN,CN,None,None
...,...,...,...,...,...,...,...,...
223,29552,1-Piperidinecarboxamide,002158-03-4,NC(=O)N1CCCCC1,C1CCNCC1,N,None,None
224,30387,Naphthalen-1-yl-urea,006950-84-1,NC(=O)Nc1cccc2ccccc12,NC1=C2C=CC=CC2=CC=C1,N,None,None
225,30511,Naphthalen-2-yl-urea,013114-62-0,NC(=O)Nc1ccc2ccccc2c1,NC1=CC=C2C=CC=CC2=C1,N,None,None
226,31339,Dimorpholamine,000119-48-2,CCCCN(CCN(CCCC)C(=O)N1CCOCC1)C(=O)N1CCOCC1,C1COCCN1,CCCCNCCN(CCCC)C(=O)N1CCOCC1,None,None


In [36]:
s=ByRow(sulfonylurea,Parents)
s

,Parent_ID,name,CAS #,Parent_SMILES,0,1
0,382,Tolbutamide,"64-77-7, 473-41-6, 000064-77-7",CCCCNC(=O)NS(=O)(=O)c1ccc(C)cc1,CC1=CC=C(C=C1)S(N)(=O)=O,CCCCN
1,1259,Chlorpropamide,"94-20-2, 000094-20-2",CCCNC(=O)NS(=O)(=O)c1ccc(Cl)cc1,NS(=O)(=O)C1=CC=C(Cl)C=C1,CCCN
2,3198,Carbutamide,"339-43-5, 000339-43-5",CCCCNC(=O)NS(=O)(=O)c1ccc(N)cc1,NC1=CC=C(C=C1)S(N)(=O)=O,CCCCN
3,3791,Sulfacarbamide,"547-44-4, 000547-44-4",NC(=O)NS(=O)(=O)c1ccc(N)cc1,NC1=CC=C(C=C1)S(N)(=O)=O,N
4,3870,Metahexamide,565-33-3,Cc1ccc(cc1N)S(=O)(=O)NC(=O)NC1CCCCC1,CC1=CC=C(C=C1N)S(N)(=O)=O,NC1CCCCC1
5,4248,Glyclopyramide,631-27-6,Clc1ccc(cc1)S(=O)(=O)NC(=O)NN1CCCC1,NS(=O)(=O)C1=CC=C(Cl)C=C1,NN1CCCC1
6,4318,Glycyclamide,"664-95-9, 000664-95-9",Cc1ccc(cc1)S(=O)(=O)NC(=O)NC1CCCCC1,CC1=CC=C(C=C1)S(N)(=O)=O,NC1CCCCC1
7,4697,Acetohexamide,968-81-0,CC(=O)c1ccc(cc1)S(=O)(=O)NC(=O)NC1CCCCC1,CC(=O)C1=CC=C(C=C1)S(N)(=O)=O,NC1CCCCC1
8,4744,Tolpentamide,1027-87-8,Cc1ccc(cc1)S(=O)(=O)NC(=O)NC1CCCC1,CC1=CC=C(C=C1)S(N)(=O)=O,NC1CCCC1
9,4750,Glyoctamide,1038-59-1,Cc1ccc(cc1)S(=O)(=O)NC(=O)NC1CCCCCCC1,CC1=CC=C(C=C1)S(N)(=O)=O,NC1CCCCCCC1


In [37]:
n=ByRow(nitrile,Parents)
n

,Parent_ID,name,CAS #,Parent_SMILES,0,1,2,3,4,5,6,7
0,101,Verapamil,"52-53-9, 152-11-4, 38321-02-7, 000052-53-9, 00...",COc1ccc(CCN(C)CCCC(C#N)(C(C)C)c2ccc(OC)c(OC)c2...,COC1=C(OC)C=C(CCN(C)CCCC(C(C)C)(C(N)=O)C2=CC(O...,None,None,None,None,None,None,None
1,521,Acetonitrile,"75-05-8, 2206-26-0, 000075-05-8",CC#N,CC(N)=O,None,None,None,None,None,None,None
2,572,2-Hydroxy-2-methylpropanenitrile,"75-86-5, 000075-86-5",CC(C)(O)C#N,CC(C)(O)C(N)=O,None,None,None,None,None,None,None
3,637,Isoaminile,77-51-0,CC(C)C(CC(C)N(C)C)(C#N)c1ccccc1,CC(C)C(CC(C)N(C)C)(C(N)=O)C1=CC=CC=C1,None,None,None,None,None,None,None
4,690,"2,2'-Azobis(2-methylpropanenitrile)","78-67-1, 000078-67-1",CC(C)(N=NC(C)(C)C#N)C#N,CC(C)(N=NC(C)(C)C(N)=O)C#N,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
799,32408,"1-(3-CYANO),2-DIBENZOYL-1-TERT-BUTYLHYDRAZINE",112426-69-4,CC(C)(C)N(NC(=O)c1ccccc1)C(=O)c1cccc(c1)C#N,CC(C)(C)N(NC(=O)C1=CC=CC=C1)C(=O)C1=CC(=CC=C1)...,None,None,None,None,None,None,None
800,32467,N(24-NO2-6CF3PH)-2(3CN-BENZO)PYRIDINAMIN,121588-32-7,[O-][N+](=O)c1cc(c(Nc2nc3ccccc3cc2C#N)c(c1)C(F...,NC(=O)C1=C(NC2=C(C=C(C=C2C(F)(F)F)[N+]([O-])=O...,None,None,None,None,None,None,None
801,32468,XDE-537 (cyhalofop-butyl),122008-78-0,CC(Oc1ccc(Oc2ccc(cc2F)C#N)cc1)C(O)=O,CC(OC1=CC=C(OC2=CC=C(C=C2F)C(N)=O)C=C1)C(O)=O,None,None,None,None,None,None,None
802,32494,N(24-NO2-6CF3 PH)-2(35CN-6ME)PYRIDINAMIN,132133-10-9,Cc1nc(Nc2c(cc(cc2C(F)(F)F)[N+]([O-])=O)[N+]([O...,CC1=NC(NC2=C(C=C(C=C2C(F)(F)F)[N+]([O-])=O)[N+...,CC1=NC(NC2=C(C=C(C=C2C(F)(F)F)[N+]([O-])=O)[N+...,None,None,None,None,None,None


In [38]:
c=ByRow(cyclic,Parents)
c

,Parent_ID,name,CAS #,Parent_SMILES,0
0,264,Biotin,"58-85-5, 4492-73-3, 000058-85-5",OC(=O)CCCCC1SCC2NC(=O)NC12,NC1CSC(CCCCC(O)=O)C1N
1,336,Niridazole,"61-57-4, 000061-57-4",[O-][N+](=O)c1cnc(s1)N1CCNC1=O,NCCNC1=NC=C(S1)[N+]([O-])=O
2,438,Trimethaphan,"68-91-7, 7187-66-8, 000068-91-7",O=C1N(Cc2ccccc2)C2C[S+]3CCCC3C2N1Cc1ccccc1,C(NC1C[S+]2CCCC2C1NCC1=CC=CC=C1)C1=CC=CC=C1
3,2431,Ethylene urea,"120-93-4, 000120-93-4",O=C1NCCN1,NCCN
4,3484,Glycoluril,"496-46-8, 36833-16-6, 68036-98-6, 000496-46-8",O=C1NC2NC(=O)NC2N1,NC1NC(=O)NC1N
5,5326,Tetrahydro-2-pyrimidinone,"1852-17-1, 001852-17-1",O=C1NCCCN1,NCCCN
6,5328,Dimethyloldihydroxyethyleneurea,1854-26-8,OCN1C(O)C(O)N(CO)C1=O,OCNC(O)C(O)NCO
7,6013,"2-Imidazolidinone, 4,5-dihydroxy-1,3-bis(metho...","3001-61-4, 68411-81-4",COCN1C(O)C(O)N(COC)C1=O,COCNC(O)C(O)NCOC
8,6300,1-(2-Hydroxyethyl)-2-imidazolidinone,3699-54-5,OCCN1CCNC1=O,NCCNCCO
9,6307,"4,5-Dihydroxytetrahydro-2H-imidazol-2-one",3720-97-6,OC1NC(=O)NC1O,NC(O)C(N)O


In [39]:
a=ByRow(amide,Parents)
a

,Parent_ID,name,CAS #,Parent_SMILES,0,1,2,3,4,5,...,10,11,12,13,14,15,16,17,18,19
0,26,Lysergide,"50-37-3, 000050-37-3",CCN(CC)C(=O)C1CN(C)C2Cc3c[nH]c4cccc(C2=C1)c34,CN1CC(C=C2C1CC1=CNC3=CC=CC2=C13)C(O)=O,CCNCC,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,41,Niclosamide,"50-65-7, 1420-04-8, 000050-65-7, 001420-04-8",Oc1ccc(Cl)cc1C(=O)Nc1ccc(cc1Cl)[N+]([O-])=O,OC(=O)C1=CC(Cl)=CC=C1O,NC1=C(Cl)C=C(C=C1)[N+]([O-])=O,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,56,Procainamide,"51-06-9, 614-39-1, 000614-39-1",CCN(CC)CCNC(=O)c1ccc(N)cc1,NC1=CC=C(C=C1)C(O)=O,CCN(CC)CCN,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,57,Nialamide,"51-12-7, 000051-12-7",O=C(CCNNC(=O)c1ccncc1)NCc1ccccc1,OC(=O)CCNNC(=O)C1=CC=NC=C1,NCC1=CC=CC=C1,OC(=O)C1=CC=NC=C1,NNCCC(=O)NCC1=CC=CC=C1,None,None,...,None,None,None,None,None,None,None,None,None,None
4,81,N-(4-Methoxyphenyl)acetamide,51-66-1,COc1ccc(NC(C)=O)cc1,CC(O)=O,COC1=CC=C(N)C=C1,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,32515,"Benzamide, N-(2-methyl-2H-benzotriazol-5-yl)-4-n",142312-67-2,Cn1nc2ccc(NC(=O)c3ccc(cc3)[N+]([O-])=O)cc2n1,OC(=O)C1=CC=C(C=C1)[N+]([O-])=O,CN1N=C2C=CC(N)=CC2=N1,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2324,32516,"Benzamide, N-(2-methyl-2H-benzotriazol-5-yl)-3,4",142312-68-3,COc1cc(cc(OC)c1OC)C(=O)Nc1ccc2nn(C)nc2c1,COC1=CC(=CC(OC)=C1OC)C(O)=O,CN1N=C2C=CC(N)=CC2=N1,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2325,32530,"1H-Indole-3-acetamide, N-[(4-MeO-Ph)Me]-5-NO2-al",149167-42-0,COc1ccc(CNC(=O)C(=O)c2c[nH]c3ccc(cc23)[N+]([O-...,OC(=O)C(=O)C1=CNC2=CC=C(C=C12)[N+]([O-])=O,COC1=CC=C(CN)C=C1,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2326,32531,"1H-Indole-3-acetamide, N-[(4-OH-Ph)Me]-5-NO2-alp",149167-56-6,Oc1ccc(CNC(=O)C(=O)c2c[nH]c3ccc(cc23)[N+]([O-]...,OC(=O)C(=O)C1=CNC2=CC=C(C=C12)[N+]([O-])=O,NCC1=CC=C(O)C=C1,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [40]:
i=ByRow(imide,Parents)
i

,Parent_ID,name,CAS #,Parent_SMILES,0,1,2,3
0,5,Phenobarbital,"50-06-6, 57-30-7, 000050-06-6",CCC1(C(=O)NC(=O)NC1=O)c1ccccc1,CCC(C(O)=O)(C(=O)NC(N)=O)C1=CC=CC=C1,None,None,None
1,8,Metharbital,"50-11-3, 000050-11-3",CCC1(CC)C(=O)NC(=O)N(C)C1=O,CCC(CC)(C(O)=O)C(=O)NC(=O)NC,CCC(CC)(C(O)=O)C(=O)N(C)C(N)=O,None,None
2,9,Mephenytoin,"50-12-4, 70989-04-7, 000050-12-4",CCC1(NC(=O)N(C)C1=O)c1ccccc1,CCC(NC(=O)NC)(C(O)=O)C1=CC=CC=C1,None,None,None
3,24,Thalidomide,"50-35-1, 841-67-8, 2614-06-4, 000050-35-1",O=C1N(C2CCC(=O)NC2=O)C(=O)c2ccccc12,OC(=O)C1=CC=CC=C1C(=O)NC1CCC(=O)NC1=O,NC(=O)C(CCC(O)=O)N1C(=O)C2=CC=CC=C2C1=O,NC(=O)CCC(N1C(=O)C2=CC=CC=C2C1=O)C(O)=O,None
4,44,Alloxan,"50-71-5, 6010-91-9, 000050-71-5",O=C1NC(=O)C(=O)C(=O)N1,NC(=O)NC(=O)C(=O)C(O)=O,None,None,None
...,...,...,...,...,...,...,...,...
563,32162,"5-PYRIMIDINECARBOXYLIC ACID, 1,2,3,4-TETRAHYDRO-",023945-44-0,OC(=O)C1C=NC(=O)NC1=O,NC(=O)\N=C/C(C(O)=O)C(O)=O,None,None,None
564,32185,"Benzoicacid,2(AcO),(25dioxo-1pyrrolid)Me",032620-72-7,CC(=O)Oc1ccccc1C(=O)OCN1C(=O)CCC1=O,CC(=O)OC1=CC=CC=C1C(=O)OCNC(=O)CCC(O)=O,None,None,None
565,32231,Tetrantoin,052094-70-9,O=C1NC(=O)C2(CCc3ccccc3C2)N1,NC(=O)NC1(CCC2=CC=CC=C2C1)C(O)=O,None,None,None
566,32253,"1(2H)-PYRIMIDINECARBOXAMIDE, 5-FLUORO-3,4-DIHYDR",060908-29-4,CN(C)C(=O)N1C=C(F)C(=O)NC1=O,CN(C)C(=O)N(\C=C(\F)C(O)=O)C(N)=O,None,None,None


In [41]:
l=ByRow(lactam,Parents)
l

,Parent_ID,name,CAS #,Parent_SMILES,0,1,2,3,4,5,6,7
0,22,Phenylbutazone,"50-33-9, 129-18-0, 60104-29-2",CCCCC1C(=O)N(N(C1=O)c1ccccc1)c1ccccc1,CCCCC(C(O)=O)C(=O)N(NC1=CC=CC=C1)C1=CC=CC=C1,None,None,None,None,None,None,None
1,38,Cephaloridine,50-59-9,OC(=O)C1=C(C[n+]2ccccc2)CSC2C(NC(=O)Cc3cccs3)C...,OC(=O)C(NC(=O)CC1=CC=CS1)C1NC(C(O)=O)=C(C[N+]2...,None,None,None,None,None,None,None
2,74,6-Propyl-2-thiouracil,51-52-5,CCCC1=NC(=S)NC(=O)C1,CCC\C(CC(O)=O)=N\C(N)=S,None,None,None,None,None,None,None
3,165,6-Methyl-2-thiouracil,"56-04-2, 000056-04-2",CC1=NC(=S)NC(=O)C1,C\C(CC(O)=O)=N\C(N)=S,None,None,None,None,None,None,None
4,168,2-Amino-6-hydroxy-1H-pyrimidin-4-one,"56-09-7, 000056-09-7",NC1=NC(=O)CC(=O)N1,NC(N)=NC(=O)CC(O)=O,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
1019,32303,"Allopurinol, 1-benzoyl-",096448-63-4,O=C(c1ccccc1)n1ncc2c1N=CNC2=O,N\C=N/C1=C(C=NN1C(=O)C1=CC=CC=C1)C(O)=O,None,None,None,None,None,None,None
1020,32304,"Allopurinol, 1-(1-oxobutyl)-",096474-05-4,CCCC(=O)n1ncc2c1N=CNC2=O,CCCC(=O)N1N=CC(C(O)=O)=C1\N=C/N,None,None,None,None,None,None,None
1021,32316,1-LAURYL-4-MeO CARBONYL-2-PYRROLODONE,101881-19-0,CCCCCCCCCCCCN1CC(CC1=O)C(=O)OC,CCCCCCCCCCCCNCC(CC(O)=O)C(=O)OC,None,None,None,None,None,None,None
1022,32378,"ACETAMIDE, N-[6,9-DIHYDRO-9-[(2-HYDROXYETHOXY)ME",110104-37-5,CC(=O)NC1=NC(=O)c2ncn(COCCO)c2N1,CC(=O)NC(=N)NC1=C(N=CN1COCCO)C(O)=O,None,None,None,None,None,None,None


In [42]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('FunctionalGroup_Exctracted_New.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
u.to_excel(writer, sheet_name='Urea',index=False)
s.to_excel(writer, sheet_name='Sulfonylurea',index=False)
n.to_excel(writer, sheet_name='Nitrile', index=False)
c.to_excel(writer,sheet_name='CyclicUrea',index=False)
a.to_excel(writer, sheet_name='Amide',index=False)
i.to_excel(writer, sheet_name='Imide',index=False)
l.to_excel(writer, sheet_name='Lactam', index=False)

# Close the Pandas Excel writer and output the Excel file.
writer.close()